In [39]:
import pandas as pd
from sklearn.cluster import k_means
from laserembeddings import Laser
import re
import spacy

In [40]:
embed = Laser()
nlp = spacy.load("en_core_web_sm")

In [55]:
df = pd.read_pickle("/home/andy/Desktop/sken_project/sentence_structure_detectore/src/resources/encoder.pkl")

In [89]:
context_noun = df.contextnoun.to_list()+df.domain_noun.to_list()+df.noun.to_list()
question = df.wquestion.to_list()
contex_verb = df.verb.to_list()+df.verv.to_list()


In [90]:
encoder={"context_noun":context_noun,"wquestions":question,"context_verb":contex_verb}

In [96]:
def clean_text(sentence):
    return re.sub(r'[^a-zA-Z ]+', '', sentence).lower()

    
def tag_data(sentence):
    text = clean_text(sentence)
    doc = nlp(text)
    resp = {"sentence":text,"sequence":''}
    for tok in doc:
        if 'W' in tok.tag_:
            if str(tok) in encoder["wquestions"]:
                resp['sequence']+="<wquestion>"
        elif 'VERB' in tok.pos_:
            if str(tok) in encoder["context_verb"]:
                resp["sequence"]+='<context_verb>'
        elif 'NOUN' in tok.pos_:
            if str(tok) in encoder['context_noun']:
                resp['sequence']+="<context_noun>"
        else:
            resp['sequence']+='<'+str(spacy.explain(tok.pos_))+'>'
    return resp
            
            
            
            

In [100]:
data_df = pd.read_excel("/home/andy/Desktop/razor_lq_data_tags.xlsx")

In [102]:
sentences = data_df['Unnamed: 0'].to_list()

In [103]:
from concurrent.futures import ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=50) as exe:
    result = list(exe.map(tag_data,sentences))

In [127]:
sequence_df = pd.DataFrame(result)
sequence_df.to_pickle('/home/andy/Desktop/sken_project/sentence_structure_detectore/src/resources/sequence.pkl')

In [112]:
mixd=pd.MultiIndex.from_frame(pd.DataFrame(result))

In [119]:
check_seq='<wquestion><context_noun><auxiliary><determiner><context_noun><context_verb><adposition>'

In [120]:
def return_idx(seq):
    result = mixd.isin([seq],level="sequence")
    matched_idx =[]
    for i,val in enumerate(result):
        if val:
            matched_idx.append(i)
    return matched_idx

In [121]:
return_idx(check_seq)

[0, 17]

'which board is your child studying in'

In [130]:
import os 

In [131]:
os.getcwd()

'/home/andy/Desktop/sken_project/sentence_structure_detectore/notebooks'